In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sys

sys.path.append("/home/dchen/BA_CH_EN/")

# Basics
import tensorflow as tf
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores
from src.models.U_net.unet import Unet

2023-06-25 18:09:56.402868: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-25 18:09:56.402900: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Data

In [3]:
# Denormed Processed Data
dat_train_denorm = ldpd.load_data_all_train_proc_denorm()
dat_test_denorm = ldpd.load_data_all_test_proc_denorm()

<IPython.core.display.Javascript object>

### 2. Data Split

In [4]:
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    dat_train_denorm
)
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(dat_test_denorm)

<IPython.core.display.Javascript object>

### 3. Building U-Net

Test with t2m lead_time = 6h

In [5]:
train_data = dat_X_train_lead_all_denorm[2][0].isel(mean_std=0)
train_data = np.expand_dims(train_data, axis=-1)

<IPython.core.display.Javascript object>

In [10]:
def pad_images(images, desired_shape=(128, 136)):
    padded_images = np.pad(
        images,
        (
            (0, 0),  # don't pad along the batch axis
            (
                (desired_shape[0] - images.shape[1]) // 2,
                (desired_shape[0] - images.shape[1]) // 2,
            ),  # pad symmetrically along the height
            (
                (desired_shape[1] - images.shape[2]) // 2,
                (desired_shape[1] - images.shape[2]) // 2,
            ),  # pad symmetrically along the width
            (0, 0),
        ),  # don't pad along the channel axis
        mode="constant",
        constant_values=0,
    )
    return padded_images

<IPython.core.display.Javascript object>

In [11]:
# pad your train and test datasets
padded_train_data = pad_images(train_data)
# padded_test_data = pad_images(test_data)

print(padded_train_data.shape)  # should print (1429, 128, 136, 1)
# print(padded_test_data.shape) # should print (357, 128, 136, 1)

(1429, 128, 136, 1)


<IPython.core.display.Javascript object>

In [13]:
# Parameters for Unet class initialization
v = "tp"  # Or any other value based on your preference
train_patches = False  # Or True if you want to train with patches

# Initialize the U-Net model
unet_model = Unet(v=v, train_patches=train_patches)

# Build the model with your training data shape
model = unet_model.build_model(dg_train_shape=padded_train_data.shape)

Downsampling block 1 shape:  (None, 64, 68, 8)
Downsampling block 2 shape:  (None, 32, 34, 16)
Downsampling block 3 shape:  (None, 16, 17, 32)
Bottleneck shape:  (None, 16, 17, 64)
Upsampling block 5 shape:  (None, 16, 17, 64)
Upsampling block 4 shape:  (None, 16, 17, 64)
Upsampling block 3 shape:  (None, 32, 34, 32)
Upsampling block 2 shape:  (None, 64, 68, 16)
Upsampling block 1 shape:  (None, 128, 136, 8)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 136, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_30 (Conv2D)             (None, 128, 136, 8)  80          ['input_4[0][

<IPython.core.display.Javascript object>